# Lesson 1 – Solutions Notebook  
Single Node Pattern: Email Subject Booster Agent

This notebook contains **one possible solution** for the exercises in:

`01_single_node_email_subject_booster.ipynb`

It is intended as a reference implementation. Your own solutions may differ in details (naming, order of operations, extra checks) but still be perfectly valid as long as the behavior matches the exercise goals.


## 1. Imports and State Definition

We extend the original `SubjectState` to include:

- `mode: str` – either `"boost"` or `"shorten"` (Exercise 1)
- `category: str` – e.g. `"project"`, `"invoice"` (Exercise 2)


In [1]:
from typing import TypedDict, Literal, Optional

Mode = Literal["boost", "shorten"]

class SubjectState(TypedDict, total=False):
    original_subject: str
    improved_subject: str
    mode: Mode           # "boost" or "shorten"
    category: str        # e.g. "project", "invoice"


## 2. Helper Functions (Optional)

We keep some small helpers to keep the main node readable.


In [2]:
def _apply_basic_boost(original: str) -> str:
    """Apply the basic boosting rules from the lesson.

    - Trim whitespace (done before calling this)
    - If very short (< 10 chars), prepend 'Update: '
    - Otherwise capitalize the first letter
    """
    if len(original) < 10:
        return f"Update: {original.capitalize()}"
    # Just capitalize first letter
    return original[0].upper() + original[1:]


def _apply_category_prefix(improved: str, category: Optional[str]) -> str:
    """Add a prefix based on category (Exercise 2)."""
    if not category:
        return improved

    if category == "invoice" and not improved.lower().startswith("invoice"):
        return f"Invoice – {improved}"

    # You could add more categories here if desired, e.g.:
    # if category == "project":
    #     return f"[Project] {improved}"

    return improved


def _apply_emoji_rules(improved: str) -> str:
    """Add emojis for certain keywords (Stretch Exercise 1)."""
    lower = improved.lower()
    # Avoid adding duplicates if already present
    if "deadline" in lower and "⏰" not in improved:
        improved += " ⏰"
    if "invoice" in lower and "💸" not in improved:
        improved += " 💸"
    return improved


def _apply_shortening(improved: str, mode: Optional[Mode], max_len: int = 40) -> str:
    """Shorten subject if mode == 'shorten' and it's too long (Exercise 1)."""
    if mode == "shorten" and len(improved) > max_len:
        return improved[: max_len - 3].rstrip() + "..."
    return improved


## 3. Final `improve_subject` Node Implementation

This version combines all exercises:

1. **Validation** (Stretch Exercise 2):
   - If `original_subject` is empty/whitespace → `"No subject provided"`.
2. **Basic boost logic** (from main lesson).
3. **Category prefix** (Exercise 2).
4. **Emoji rules** (Stretch Exercise 1).
5. **Shorten vs boost behavior** (Exercise 1) – controlled by `mode`.


In [3]:
def improve_subject(state: SubjectState) -> SubjectState:
    """Full solution node for Lesson 1.

    Implements:
    - Validation for empty original_subject
    - Basic subject boosting
    - Category-based prefixing
    - Emoji enhancement
    - Shorten vs boost mode
    """
    raw_original = state.get("original_subject", "")
    original = raw_original.strip()

    # 1. Validation for empty or whitespace-only subject
    if not original:
        state["improved_subject"] = "No subject provided"
        return state

    # 2. Basic boost
    improved = _apply_basic_boost(original)

    # 3. Category prefix
    category = state.get("category")
    improved = _apply_category_prefix(improved, category)

    # 4. Emoji rules
    improved = _apply_emoji_rules(improved)

    # 5. Shorten vs boost mode
    mode = state.get("mode", "boost")  # default to boost
    improved = _apply_shortening(improved, mode)  # Exercise 1

    state["improved_subject"] = improved
    return state


## 4. Wiring Up the LangGraph App

We now wrap this node into a `StateGraph`, just like in the lesson notebook.


In [4]:
from langgraph.graph import StateGraph

graph = StateGraph(SubjectState)
graph.add_node("improve_subject", improve_subject)
graph.set_entry_point("improve_subject")
graph.set_finish_point("improve_subject")  # single-node graph

app = graph.compile()

## 5. Example Calls

Below are several test cases that illustrate the behavior required by the exercises.


In [5]:
# 5.1 Basic boost
state: SubjectState = {
    "original_subject": "meeting",
    "improved_subject": "",
    "mode": "boost",
    "category": "project",
}
app.invoke(state)

{'original_subject': 'meeting',
 'improved_subject': 'Update: Meeting',
 'mode': 'boost',
 'category': 'project'}

In [6]:
# 5.2 Invoice category + emojis + boost
state_invoice: SubjectState = {
    "original_subject": "invoice 1234 due soon",
    "improved_subject": "",
    "mode": "boost",
    "category": "invoice",
}
app.invoke(state_invoice)

{'original_subject': 'invoice 1234 due soon',
 'improved_subject': 'Invoice 1234 due soon 💸',
 'mode': 'boost',
 'category': 'invoice'}

In [7]:
# 5.3 Shorten mode (long subject)
state_long: SubjectState = {
    "original_subject": "very long subject line about a project deadline next week",
    "improved_subject": "",
    "mode": "shorten",
    "category": "project",
}
app.invoke(state_long)

{'original_subject': 'very long subject line about a project deadline next week',
 'improved_subject': 'Very long subject line about a projec...',
 'mode': 'shorten',
 'category': 'project'}

In [8]:
# 5.4 Empty subject validation
state_empty: SubjectState = {
    "original_subject": "   ",
    "improved_subject": "",
    "mode": "boost",
    "category": "invoice",
}
app.invoke(state_empty)

{'original_subject': '   ',
 'improved_subject': 'No subject provided',
 'mode': 'boost',
 'category': 'invoice'}

You can modify these examples or add more to experiment further.

This notebook provides one clean, fully worked solution for Lesson 1.  
In your teaching or self-study workflow, you can keep this notebook separate from the main lesson notebook to avoid spoilers for students.
